In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv('C:/Users/Kate/Downloads/Product_Analysis_data2.csv')

In [7]:
# Convert time column to datetime type
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])
df['first_purchase_time'] = pd.to_datetime(df['first_purchase_time'])
df['cohort_week'] = pd.to_datetime(df['cohort_week'])
df['cohort_week'] = pd.to_datetime(df['cohort_week'])
df['cohort_month'] = df['cohort_week'].dt.to_period('M')

In [8]:
# Turn string values into floats
df = df.drop(['product_id'], axis=1)
df['purchase'] = 2.49
#df.head()

In [10]:
# Simple LTV: ARPU (Cумарний прирбуток за період) / CQ (Кіл-ть клієнтів за період)
# arpu = df['purchase'].sum() / df['user_id'].nunique()
df['arpu_w'] = df.groupby('cohort_week')['purchase'].transform('sum') / df.groupby('cohort_week')['purchase'].transform('count')
df['users_w'] = df.groupby('cohort_week')['user_id'].transform('nunique')
df['ltv_w1'] = df['arpu_w'] / df['users_w']
df.head()

,user_id,event_timestamp,first_purchase_time,cohort_week,lifetime_weeks,cohort_month,purchase,arpu_w,users_w,ltv_w1
0,6f8592c0-fb4c-47dc-9344-424eb5d88a4b,2023-10-13 09:54:02,2023-10-13 09:54:02,2023-10-08,0,2023-10,2.49,2.49,1,2.490000
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-10-14 04:45:14,2023-03-25 04:44:40,2023-03-19,29,2023-03,2.49,2.49,84,0.029643
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 04:44:40,2023-03-25 04:44:40,2023-03-19,0,2023-03,2.49,2.49,84,0.029643
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-01 04:44:16,2023-03-25 04:44:40,2023-03-19,1,2023-03,2.49,2.49,84,0.029643
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-08 04:45:55,2023-03-25 04:44:40,2023-03-19,2,2023-03,2.49,2.49,84,0.029643


In [12]:
# The same but per month period
df['arpu_m'] = df.groupby('cohort_month')['purchase'].transform('sum') / df.groupby('cohort_month')['purchase'].transform('count')
df['users_m'] = df.groupby('cohort_month')['user_id'].transform('nunique')
df['ltv_m1'] = df['arpu_m'] / df['users_m']
#df.head()

In [13]:
# Simple LTV2: ARPU (Average revenue per user) * CL (Customer Lifetime)
df['ltv_w2'] = df['arpu_w'] * df['lifetime_weeks']

In [16]:
# Average LTV:  CLV (Customer Lifetime Value) .Mean
#CLV = Average Purchase Value (APV) x Average Purchase Frequency (APF) x Average Customer Lifespan (ACL).
df['acl'] = df.groupby('cohort_week')['lifetime_weeks'].transform('mean')
df['apv'] = df.groupby(['cohort_week', 'user_id'])['purchase'].transform('mean')
df['apf'] = df.groupby(['cohort_week', 'user_id'])['purchase'].transform('count')

df['clv'] = df['apv'] * df['apf'] * df['acl']
df['ltv_w3'] = df.groupby('cohort_week')['clv'].transform('mean')
df['ltv_w5'] = df['clv'] * df['acl']
df.head()

,user_id,event_timestamp,first_purchase_time,cohort_week,lifetime_weeks,cohort_month,purchase,arpu_w,users_w,ltv_w1,arpu_m,users_m,ltv_m1,ltv_w2,acl,apv,apf,clv,ltv_w3,ltv_w5
0,6f8592c0-fb4c-47dc-9344-424eb5d88a4b,2023-10-13 09:54:02,2023-10-13 09:54:02,2023-10-08,0,2023-10,2.49,2.49,1,2.490000,2.49,1,2.490000,0.00,0.000000,2.49,1,0.000000,0.000000,0.000000
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-10-14 04:45:14,2023-03-25 04:44:40,2023-03-19,29,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,72.21,8.795414,2.49,32,700.818624,389.305055,6163.990264
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 04:44:40,2023-03-25 04:44:40,2023-03-19,0,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,0.00,8.795414,2.49,32,700.818624,389.305055,6163.990264
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-01 04:44:16,2023-03-25 04:44:40,2023-03-19,1,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,2.49,8.795414,2.49,32,700.818624,389.305055,6163.990264
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-08 04:45:55,2023-03-25 04:44:40,2023-03-19,2,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,4.98,8.795414,2.49,32,700.818624,389.305055,6163.990264


In [19]:
# Average LTV: ARPU / (1 - Churn Rate)
# Churn Rate:  (total_users - lost_users) / total_users

# Total users per cohort week
total_users = df.groupby('cohort_week')['user_id'].transform('nunique')

# Lost users per cohort week
max_lifetime_weeks = df['lifetime_weeks'].max()
lost_users = df[df['lifetime_weeks'] == max_lifetime_weeks].groupby('cohort_week')['user_id'].transform('nunique')

# Churn rate
churn_rate = (total_users - lost_users) / total_users

# Retention rate
retention_rate = 1 - churn_rate

# LTV
df['ltv_w4'] = df['arpu_w'] / (1 - churn_rate) #.fillna(0)
df.head().drop_duplicates()

,user_id,event_timestamp,first_purchase_time,cohort_week,lifetime_weeks,cohort_month,purchase,arpu_w,users_w,ltv_w1,arpu_m,users_m,ltv_m1,ltv_w2,acl,apv,apf,clv,ltv_w3,ltv_w5,ltv_w4
0,6f8592c0-fb4c-47dc-9344-424eb5d88a4b,2023-10-13 09:54:02,2023-10-13 09:54:02,2023-10-08,0,2023-10,2.49,2.49,1,2.490000,2.49,1,2.490000,0.00,0.000000,2.49,1,0.000000,0.000000,0.000000,NaN
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-10-14 04:45:14,2023-03-25 04:44:40,2023-03-19,29,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,72.21,8.795414,2.49,32,700.818624,389.305055,6163.990264,NaN
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 04:44:40,2023-03-25 04:44:40,2023-03-19,0,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,0.00,8.795414,2.49,32,700.818624,389.305055,6163.990264,NaN
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-01 04:44:16,2023-03-25 04:44:40,2023-03-19,1,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,2.49,8.795414,2.49,32,700.818624,389.305055,6163.990264,NaN
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-08 04:45:55,2023-03-25 04:44:40,2023-03-19,2,2023-03,2.49,2.49,84,0.029643,2.49,291,0.008557,4.98,8.795414,2.49,32,700.818624,389.305055,6163.990264,NaN


In [ ]:
#df.to_csv (r'C:/Users/Kate/Downloads/LTV_Results.csv', header=True)